# Cornerstone Project - Eric Rodrigues de Carvalho

This notebook is a series of codes that serves as one of the primary tools to execute my cornerstone project as an undergraduated student called "SUPER-RESOLUÇÃO DE IMAGENS EM
TOMOGRAFIA COMPUTADORIZADA DE BAIXA
DOSAGEM: COMPARAÇÃO DE MÉTODOS DE
APRENDIZADO PROFUNDO" (SUPER-RESOLUTION OF IMAGES IN
COMPUTED TOMOGRAPHY OF LOW
DOSAGE: COMPARISON OF METHODS OF
DEEP LEARNING).

The main goals of this notebook are:

1. Convert files from HDF5 (the source file) to actual PNG images.
2. Convert sinogram representation to actual and human-readable CT-scanned images.
3. Convert SRCNN model to ONNX-based in order to run in chaiNNer application.
4. After using chaiNNer as specified in the thesis, execute metrics in a subset of images to evaluate DL models performance in both no-reference and full-reference metrics and also its runtime.
5. Plot examples for visual comparision.


In [ ]:


# Un-comment if you wish to use Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

## HDF to BMP and Sinogram to human-readable CT

Download the zip containing the datasets at https://zenodo.org/records/3384092. For this cornerstone project, only ground_truth_test.zip and observation_test.zip are needed. After unzipping in the folder of preference, set your dataset_path and the hdf5_folders based on your own desired folders' structure

This script processes two datasets (ground truth and observation) from HDF5 files, typically used in medical imaging or computed tomography reconstruction scenarios. It converts these data into BMP image format and saves them into pre-defined directories.

Key steps in the code:

Directory Creation: The script ensures that the directories for storing the images (both ground truth and observation) exist. If they don't, it creates them using the create_dir function.

Ground Truth and Observation Processing: It reads HDF5 files containing arrays of image data. The ground truth data is directly saved as images after rotating them. The observation data, on the other hand, undergoes more complex processing: it performs a reconstruction using the iradon function (Filtered Back Projection) and crops the resulting image to a standard size before saving.

Error Handling: The code handles potential errors in reading files. If an error occurs while processing observation data, the corresponding ground truth image is deleted to maintain data consistency.

FBP Reconstruction: The observation data is reconstructed using the iradon function from the skimage.transform library. This method applies a transformation called Filtered Back Projection, commonly used in medical imaging to reconstruct cross-sectional images from projection data (as in CT scans).


In [ ]:
"""
This script processes medical imaging data stored in HDF5 format. Specifically, it reads the
ground truth and observation test datasets, processes them into images, and saves them in BMP format.

The script does the following:
1. Creates directories for saving ground truth and observation images if they don't exist.
2. Reads ground truth and observation HDF5 files.
3. For each file:
   - Ground truth images are rotated and saved directly.
   - Observation data is used to reconstruct images using the Filtered Back Projection (FBP) technique,
     crop the result, and save the reconstructed images.
4. If an error occurs in reading the observation file, the corresponding ground truth image is deleted to
   ensure data consistency.

Dependencies:
- h5py: For reading HDF5 files.
- numpy: For numerical operations, including image manipulation.
- matplotlib: For saving images in BMP format.
- skimage: For performing the Filtered Back Projection (FBP) reconstruction.

"""

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import iradon

# Path to the dataset folder containing HDF5 files
dataset_path = 'your/path/here'

# List of folder names within the dataset containing various datasets (train, test, validation)
hdf5_folders = [
    'ground_truth_train',
    'ground_truth_test',
    'ground_truth_validation',
    'observation_train',
    'observation_test',
    'observation_validation',
]

# Function to create directories if they do not exist


def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


# Process ground truth test data by saving images in a specific directory
ground_truth_test_images_dir = os.path.join(
    dataset_path, 'ground_truth_test_images')
create_dir(ground_truth_test_images_dir)

# Process observation test data by saving images in a specific directory
observation_test_images_dir = os.path.join(
    dataset_path, 'observation_test_images')
create_dir(observation_test_images_dir)

# Get a sorted list of ground truth and observation HDF5 files
ground_truth_files = sorted(os.listdir(
    os.path.join(dataset_path, hdf5_folders[1])))
observation_files = sorted(os.listdir(
    os.path.join(dataset_path, hdf5_folders[4])))

# Initialize counters for naming the saved image files
ground_truth_counter = 0
observation_counter = 0

# Process both ground truth and observation HDF5 files
for gt_file_name, obs_file_name in zip(ground_truth_files, observation_files):
    gt_file_path = os.path.join(dataset_path, hdf5_folders[1], gt_file_name)
    obs_file_path = os.path.join(dataset_path, hdf5_folders[4], obs_file_name)

    # Try opening and processing the ground truth HDF5 file
    try:
        with h5py.File(gt_file_path, 'r') as gt_hdf5_file:
            for j in range(len(gt_hdf5_file['data'])):
                # Extract the ground truth image data
                gt_data = gt_hdf5_file['data'][j]

                # Save the ground truth image after rotating 90 degrees
                gt_image_name = f'ground_truth_test_{
                    ground_truth_counter:04d}.bmp'
                plt.imsave(os.path.join(ground_truth_test_images_dir,
                           gt_image_name), arr=np.rot90(gt_data, k=1), cmap='gray')
                ground_truth_counter += 1

        # Try opening and processing the corresponding observation HDF5 file
        try:
            with h5py.File(obs_file_path, 'r') as obs_hdf5_file:
                for j in range(len(obs_hdf5_file['data'])):
                    # Extract observation data and transpose for FBP reconstruction
                    obs_data = obs_hdf5_file['data'][j]
                    obs_data = np.transpose(obs_data)

                    # Generate angles for FBP reconstruction
                    theta = np.linspace(0., 180., max(
                        obs_data.shape), endpoint=False)

                    # Perform FBP (Filtered Back Projection) reconstruction
                    reconstruction_fbp = iradon(
                        obs_data, theta=theta, filter_name='ramp')

                    # Crop the reconstructed image to a fixed size (362x362)
                    crop_height = min(362, reconstruction_fbp.shape[0])
                    crop_width = min(362, reconstruction_fbp.shape[1])
                    start_row = (
                        reconstruction_fbp.shape[0] - crop_height) // 2
                    start_col = (reconstruction_fbp.shape[1] - crop_width) // 2
                    reconstruction_fbp = reconstruction_fbp[start_row:start_row +
                                                            crop_height, start_col:start_col + crop_width]

                    # Save the observation image
                    obs_image_name = f'observation_test_{
                        observation_counter:04d}.bmp'
                    plt.imsave(os.path.join(observation_test_images_dir,
                               obs_image_name), arr=reconstruction_fbp, cmap='gray')
                    observation_counter += 1

        except OSError as e:
            # Error handling for observation files
            print(f"Error opening {obs_file_path}: {e}")
            # Remove the corresponding ground truth image if observation fails
            os.remove(os.path.join(ground_truth_test_images_dir, gt_image_name))
            ground_truth_counter -= 1  # Adjust the counter to maintain consistency

    except OSError as e:
        # Error handling for ground truth files
        print(f"Error opening {gt_file_path}: {e}")
        # Remove the corresponding observation file if ground truth processing fails
        if os.path.exists(obs_file_path):
            os.remove(obs_file_path)

## SRCNN as an ONNX framework


This code defines a Super-Resolution Convolutional Neural Network (SRCNN) designed for single-channel (grayscale) images. The SRCNN model is a simple but effective deep learning model that performs image super-resolution by mapping low-resolution images to high-resolution counterparts through three convolutional layers. This script loads pre-trained weights for the SRCNN, sets the model to evaluation mode, and exports it to the ONNX format.

Key components of the code:

Model Definition: The SRCNN class defines a neural network with three convolutional layers. Each layer applies a different level of feature extraction and image refinement.

The first layer uses 64 filters to extract features.
The second layer reduces the feature space to 32 dimensions.
The third layer outputs the reconstructed high-resolution image.
ReLU activation is applied after the first two convolutional layers to introduce non-linearity.
Loading Pre-Trained Weights: The script loads pre-trained weights (from a .pth file) into the SRCNN model. These weights are assumed to have been trained on an external dataset.

Evaluation Mode: Once the model is loaded with weights, it's set to evaluation mode. This ensures that certain layers (like dropout or batch normalization, if they existed) behave properly during inference.

Export to ONNX: The model is exported to the ONNX format using the torch.onnx.export function, with a dummy input tensor that simulates a grayscale (single-channel) image


In [ ]:
!pip install onnx

In [ ]:
"""
This script defines and exports a Super-Resolution Convolutional Neural Network (SRCNN) model to the ONNX format.
SRCNN is designed for single-channel (grayscale) image input, and this implementation is based on the architecture
outlined in the SRCNN paper for image super-resolution.

The script includes the following steps:
1. Define the SRCNN model architecture.
2. Load pre-trained weights (state_dict) into the model.
3. Set the model to evaluation mode.
4. Export the model to ONNX format using a dummy single-channel image input for format specification.

Dependencies:
- torch: For defining and loading the SRCNN model and its weights.
- torch.onnx: For exporting the PyTorch model to ONNX format.

"""


import torch
from torch import nn
import torch.onnx

# Define the SRCNN model class for single-channel input images


class SRCNN(nn.Module):
    def __init__(self, num_channels=1):
        """
        Initializes the SRCNN model.

        Args:
            num_channels (int): Number of input channels (default is 1 for grayscale images).
        """
        super(SRCNN, self).__init__()
        # First convolutional layer with 64 filters, kernel size 9x9, and padding to keep output size same as input
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)

        # Second convolutional layer with 32 filters, kernel size 5x5, and padding to maintain spatial dimensions
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)

        # Third convolutional layer that outputs the final single-channel (grayscale) image
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)

        # ReLU activation function applied after the first two convolutional layers
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """
        Defines the forward pass through the network.

        Args:
            x (torch.Tensor): Input image tensor of shape [batch_size, num_channels, height, width].

        Returns:
            torch.Tensor: The output high-resolution image tensor after passing through the network.
        """
        # Pass input through the first convolutional layer followed by ReLU activation
        x = self.relu(self.conv1(x))

        # Pass through the second convolutional layer followed by ReLU activation
        x = self.relu(self.conv2(x))

        # Pass through the third convolutional layer to generate the final output image
        x = self.conv3(x)
        return x


# Create an instance of the SRCNN model for single-channel (grayscale) input
model = SRCNN(num_channels=1)

# Load the pre-trained model weights from the .pth file
# Adjust the path to where the pre-trained weights are located
state_dict = torch.load('/your/path/here/srcnn_x2.pth')
model.load_state_dict(state_dict)  # Load the weights into the model

# Set the model to evaluation mode to ensure correct behavior during inference (e.g., disabling dropout if present)
model.eval()

# Export the model to ONNX format
torch.onnx.export(
    model,                      # The model to be exported
    # Dummy input of size [batch_size, num_channels, height, width] for grayscale images
    torch.randn(1, 1, 224, 224),
    "/your/path/here/srcnn_x2.onnx",    # Path where the ONNX model will be saved
    export_params=True,          # Whether to store the trained weights inside the ONNX model
    verbose=False,               # Whether to print detailed logs during export
    opset_version=10             # ONNX opset version to ensure compatibility
)

## Super-Resolution Model Evaluation Process


This script is designed to evaluate the performance of multiple image super-resolution models by calculating several image quality metrics between the super-resolved images generated by these models and the ground truth high-resolution images. The models being evaluated include DAT, ESRGAN, HAT, SRCNN, and SwinIR, all of which aim to reconstruct high-quality images from lower-resolution observations.

#### 1. **Input Preparation**

- **Observation Images**: These are the low-resolution test images that will be super-resolved by the different models. The observation images are stored in a designated directory and follow a structured naming convention.
- **Ground Truth Images**: The corresponding high-resolution images are used as references for comparison. These images are essential for calculating the quality metrics and are stored in a separate directory, also following a structured naming format.

#### 2. **Super-Resolved Images from Models**

- Each model has its own folder where the super-resolved images generated by that model are stored. These images are compared against the ground truth to determine how well each model reconstructs the high-resolution details from the low-resolution observations.

#### 3. **Metrics Calculation**

To assess the quality of the super-resolved images, the following metrics are calculated:

- **PSNR (Peak Signal-to-Noise Ratio)**: This measures how closely the super-resolved image resembles the ground truth in terms of pixel intensity.
- **SSIM (Structural Similarity Index)**: SSIM evaluates how similar the structure of the image is compared to the ground truth, focusing on contrast, luminance, and texture.
- **LPIPS (Learned Perceptual Image Patch Similarity)**: LPIPS assesses the perceptual quality of the image, accounting for how humans perceive image differences, using deep learning.
- **NIQE (Natural Image Quality Evaluator)**: This is a no-reference metric, meaning it doesn’t require a ground truth image. It assesses the quality of the super-resolved image based on statistical properties.
- **NRQM (No-Reference Quality Metric)**: Like NIQE, this is a no-reference metric but focuses on perceptual image quality without requiring a reference image.
- **PI (Perceptual Index)**: This is a combination of the NIQE and NRQM scores and provides an overall perceptual quality index for the image.
  The metric values, along with the computation times, are recorded for analysis.

#### 4. **Processing Flow**

- **Preprocessing**: The observation and ground truth images are loaded from their respective directories and converted to arrays for further processing.
- **Metric Computation**: For each pair of images (observation/model output vs. ground truth), the defined metrics are computed. In some cases, like LPIPS, NIQE, and NRQM, the images are first converted to PyTorch tensors.
- **Result Storage**: The calculated metric values, along with the corresponding image IDs and model names, are saved to a structured list. Additionally, the runtime for each metric calculation is recorded for performance analysis.

#### 5. **Data Export**

- Once all images and models are processed, the results are compiled into a pandas DataFrame, which is then exported as a CSV file. This CSV contains detailed information on each metric for each image and model, providing a comprehensive evaluation of the models’ performance.

#### 6. **Output Analysis**

- The final DataFrame allows for further analysis of how each model performs across different images and metrics. By analyzing this data, one can determine which models excel in different aspects of super-resolution, such as perceptual quality, structural accuracy, or overall similarity to the ground truth.

This evaluation process not only provides quantitative insights into each model's performance but also highlights trade-offs between various metrics, allowing for informed decisions when choosing a model for specific tasks.


In [ ]:
!pip install lpips
!pip install pyiqa

In [ ]:
"""
This script evaluates the performance of different image super-resolution models
by calculating various image quality metrics, such as PSNR, SSIM, LPIPS, NIQE, NRQM,
and Perceptual Index (PI), between the super-resolved images generated by the models
and the ground truth high-resolution images.
"""

import os
import glob
import numpy as np
from PIL import Image
import skimage.metrics
import torch
from lpips import LPIPS
import pyiqa
import pandas as pd
import time

# Paths to the folders containing the observation images, ground truth images, and model-generated images.
observation_path = 'your/path/here/Dataset/observation_test_images'
ground_truth_path = 'your/path/here/Dataset/ground_truth_test_images'
model_paths = {
    'DAT': 'your/path/here/Images/Observation/DAT',
    'ESRGAN': 'your/path/here/Images/Observation/ESRGAN',
    'HAT': 'your/path/here/Images/Observation/HAT',
    'SRCNN': 'your/path/here/Images/Observation/SRCNN',
    'SwinIR': 'your/path/here/Images/Observation/SwinIR'
}

# Define the metrics to calculate for evaluation
metrics = ['PSNR', 'SSIM', 'LPIPS', 'NIQE', 'NRQM', 'PI']

# Initialize the LPIPS model for perceptual similarity (AlexNet backbone)
lpips_model = LPIPS(net='alex')

# Initialize the NIQE and NRQM models using pyiqa for no-reference image quality assessments
niqe_model = pyiqa.create_metric('niqe')
nrqm_model = pyiqa.create_metric('nrqm')


def pil_to_tensor(image):
    """
    Converts a PIL image to a PyTorch tensor.

    Args:
        image (PIL.Image): Input PIL image.

    Returns:
        torch.Tensor: Image converted to a 4D tensor with shape (1, C, H, W).
    """
    image = np.array(image).astype(np.float32) / \
        255.0  # Normalize to [0, 1] range
    image = torch.tensor(image).permute(2, 0, 1)  # Change to [C, H, W] format
    image = image.unsqueeze(0)  # Add batch dimension [1, C, H, W]
    return image


def calculate_pi(niqe, nrqm):
    """
    Calculates the Perceptual Index (PI) metric based on NIQE and NRQM scores.

    PI = (10 - NRQM) / 2 + NIQE / 2

    Args:
        niqe (float): NIQE score of the image.
        nrqm (float): NRQM score of the image.

    Returns:
        float: Perceptual Index score.
    """
    return (10 - nrqm) / 2 + niqe / 2


def calculate_metrics(image1, image2):
    """
    Calculates multiple image quality metrics (PSNR, SSIM, LPIPS, NIQE, NRQM, PI)
    between two images.

    Args:
        image1 (np.array): Ground truth image as a NumPy array.
        image2 (np.array): Test or model-generated image as a NumPy array.

    Returns:
        dict: A dictionary with metric names as keys and tuples of (metric_value, runtime) as values.
    """
    metrics_data = {}

    # PSNR (Peak Signal-to-Noise Ratio)
    start_time = time.time()
    psnr = skimage.metrics.peak_signal_noise_ratio(image1, image2)
    metrics_data['PSNR'] = (psnr, time.time() - start_time)

    # SSIM (Structural Similarity Index)
    start_time = time.time()
    ssim = skimage.metrics.structural_similarity(
        image1, image2, channel_axis=-1)
    metrics_data['SSIM'] = (ssim, time.time() - start_time)

    # Convert images to tensors for LPIPS, NIQE, and NRQM calculation
    observation_tensor = pil_to_tensor(observation_img)
    ground_truth_tensor = pil_to_tensor(ground_truth_img)

    # LPIPS (Learned Perceptual Image Patch Similarity)
    start_time = time.time()
    lpips_score = lpips_model(ground_truth_tensor, observation_tensor).item()
    metrics_data['LPIPS'] = (lpips_score, time.time() - start_time)

    # NIQE (Natural Image Quality Evaluator)
    start_time = time.time()
    niqe_score = niqe_model(observation_tensor).item()
    metrics_data['NIQE'] = (niqe_score, time.time() - start_time)

    # NRQM (No-Reference Quality Metric)
    start_time = time.time()
    nrqm_score = nrqm_model(observation_tensor).item()
    metrics_data['NRQM'] = (nrqm_score, time.time() - start_time)

    # Perceptual Index (PI)
    start_time = time.time()
    pi_score = calculate_pi(niqe_score, nrqm_score)
    metrics_data['PI'] = (pi_score, time.time() - start_time)

    return metrics_data


# Get a list of all observation images in the dataset and sort by filename to ensure consistency
observation_images = glob.glob(os.path.join(observation_path, '*.bmp'))
observation_images.sort()

# Select the first 150 observation images
selected_observation_images = observation_images[:150]

# Extract the image IDs from the filenames (assuming filenames follow a specific format)
image_ids = [int(os.path.basename(image).split('.')[0].split('_')[-1])
             for image in selected_observation_images]

# Initialize an empty list to store results
data = []

# Process the observation images and calculate metrics
for observation_image, observation_number in zip(selected_observation_images, image_ids):
    # Find the corresponding ground truth image
    ground_truth_image = os.path.join(
        ground_truth_path,
        f'ground_truth_test_{observation_number:04d}.bmp'
    )

    # Open the observation and ground truth images as PIL images
    observation_img = Image.open(observation_image)
    ground_truth_img = Image.open(ground_truth_image)

    # Convert the images to NumPy arrays
    observation_array = np.array(observation_img)
    ground_truth_array = np.array(ground_truth_img)

    # Calculate metrics for the observation image
    observation_metrics = calculate_metrics(
        ground_truth_array, observation_array)
    for metric, (value, runtime) in observation_metrics.items():
        data.append([observation_number, 'Observation',
                    metric, value, runtime])

# Process images from each model and calculate metrics
for model, model_path in model_paths.items():
    for observation_number in image_ids:
        # Find the corresponding model-generated and ground truth images
        model_image = os.path.join(
            model_path,
            f'observation_test_{observation_number:04d}.bmp'
        )
        ground_truth_image = os.path.join(
            ground_truth_path,
            f'ground_truth_test_{observation_number:04d}.bmp'
        )

        # Open the model and ground truth images as PIL images
        observation_img = Image.open(model_image)
        ground_truth_img = Image.open(ground_truth_image)

        # Convert the images to NumPy arrays
        observation_array = np.array(observation_img)
        ground_truth_array = np.array(ground_truth_img)

        # Calculate metrics for the model-generated image
        model_metrics = calculate_metrics(
            ground_truth_array, observation_array)
        for metric, (value, runtime) in model_metrics.items():
            data.append([observation_number, model, metric, value, runtime])

# Convert the collected data into a DataFrame for analysis and export
df = pd.DataFrame(
    data, columns=['Image_ID', 'Model', 'Metric', 'Value', 'Runtime'])

# Save the DataFrame to a CSV file for further use
df.to_csv('your/path/here/metrics_results.csv', index=False)

# Print the DataFrame to view the results
print(df)

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('your/path/here/metrics_results.csv')

# Group by 'Model' and 'Metric' and calculate the mean and standard deviation of 'Value'
aggregated_df = df.groupby(['Model', 'Metric']).agg(
    {'Value': ['mean', 'std']}
)

# Print the aggregated DataFrame
print(aggregated_df)

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('your/path/here/metrics_results.csv')

# Group by 'Model' and 'Metric' and calculate the mean and standard deviation of 'Value'
aggregated_df = df.groupby(['Model', 'Metric']).agg(
    {'Value': ['mean', 'std']}
)

# Get the unique metrics
metrics = df['Metric'].unique()

# Initialize the dictionary to store the best models
best_results = {}

# Iterate over each metric
for metric in metrics:
    # If the metric is PSNR, SSIM, or NRQM, find the model with the highest mean value
    if metric in ['PSNR', 'SSIM', 'NRQM']:
        best_model = aggregated_df.loc[(
            slice(None), metric), ('Value', 'mean')].idxmax()[0]
    # Otherwise, find the model with the lowest mean value
    else:
        best_model = aggregated_df.loc[(
            slice(None), metric), ('Value', 'mean')].idxmin()[0]
    # Store the best model for the current metric
    best_results[metric] = best_model

# Print the best models for each metric
print('Best results:')
for metric, model in best_results.items():
    print(f'{metric}: {model}')

In [ ]:
# Read the CSV file
df = pd.read_csv('metrics_results (2).csv')

# Group by 'Model' and 'Metric' and calculate the mean and standard deviation of 'Value'
aggregated_df = df.groupby(['Model', 'Metric']).agg(
    {'Value': ['mean', 'std']}
)

# Print the aggregated DataFrame
print(aggregated_df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/metrics_results (2).csv')

# Set up the grid for 3 rows and 2 columns
fig, axes = plt.subplots(3, 2, figsize=(12, 18))
axes = axes.flatten()

# Define the metrics
metrics = df['Metric'].unique()

# Loop through each metric
for i, metric in enumerate(metrics):
    # Select the axis
    ax = axes[i]

    # Filter the data for the current metric
    data = df[df['Metric'] == metric]

    # Create the boxplot
    boxplot = sns.boxplot(x='Model', y='Value', data=data, ax=ax)

    # Customize colors for specific models
    for j, artist in enumerate(ax.patches):
        # Extract the corresponding model for the current box
        model = data['Model'].unique()[j]

        if model == 'ESRGAN':
            artist.set_facecolor('lightgreen')

    # Set titles and labels
    ax.set_title(f'Diagrama de Caixa da {metric}')
    ax.set_xlabel('Modelo')
    ax.set_ylabel('Valor')

# Adjust layout
plt.tight_layout()
plt.show()

## Sample plot for visual comparision


This script visualizes the super-resolved images generated by various models (DAT, ESRGAN, HAT, SRCNN, and SwinIR) alongside the low-resolution observation image and the corresponding ground truth high-resolution image. The process involves loading images from pre-specified directories, displaying them in a side-by-side comparison, and saving these visualizations as high-resolution PNG files.

Steps:

Define Paths: Paths to the observation, ground truth, and model-generated images are specified. Each model has its own directory where its super-resolved images are stored.
Image Loading: The observation image and the ground truth image are located and loaded. Similarly, for each model, the super-resolved images corresponding to the observation are loaded.
Visualization: Two sets of visualizations are created:
The ground truth image is saved individually.
The observation and model images are displayed in a 2x3 grid for comparison, with the observation image and each model’s output in its own subplot.
Saving and Displaying: The plots are saved as PNG files with high resolution, and the second plot (observation + model images) is displayed for immediate inspection.
This approach allows for a clear visual comparison of how different models reconstruct the super-resolved images relative to the ground truth.


In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image


def visualize_model_comparisons(image_id):
    """
    Visualizes the super-resolved images generated by different models, comparing them
    with the observation image and the ground truth image. Saves the visualizations
    as PNG files.

    Parameters:
    -----------
    image_id : int
        The ID of the image to visualize (used for both observation and ground truth images).

    Process:
    1. Load the observation image and ground truth image.
    2. Load the super-resolved images from five different models: DAT, ESRGAN, HAT, SRCNN, and SwinIR.
    3. Display the ground truth image individually and save it.
    4. Create a 2x3 grid to visualize the observation image alongside the super-resolved images from each model.
    5. Save the comparison as a PNG image and display the plot.
    """
    # Define the paths to the different folders
    observation_path = 'your/path/here/Dataset/observation_test_images'
    ground_truth_path = 'your/path/here/Dataset/ground_truth_test_images'
    model_paths = {
        'DAT': 'your/path/here/Images/DAT',
        'ESRGAN': 'your/path/here/Images/ESRGAN',
        'HAT': 'your/path/here/Images/HAT',
        'SRCNN': 'your/path/here/Images/SRCNN',
        'SwinIR': 'your/path/here/Images/SwinIR'
    }

    # Find and load the observation image
    observation_image = os.path.join(
        observation_path, f'observation_test_{image_id:04d}.bmp')
    observation_img = Image.open(observation_image)

    # Find and load the ground truth image
    ground_truth_image = os.path.join(
        ground_truth_path, f'ground_truth_test_{image_id:04d}.bmp')
    ground_truth_img = Image.open(ground_truth_image)

    # Find and load the model-generated images
    model_images = {}
    for model, model_path in model_paths.items():
        model_image = os.path.join(
            model_path, f'observation_test_{image_id:04d}.bmp')
        model_images[model] = Image.open(model_image)

    # Save the ground truth image individually
    fig_gt, ax_gt = plt.subplots(figsize=(5, 5))
    ax_gt.imshow(ground_truth_img)
    ax_gt.set_title('Ground Truth', fontsize=14)
    ax_gt.axis('off')
    plt.savefig(f'ground_truth_image_{
                image_id}.png', dpi=300, bbox_inches='tight')
    plt.close(fig_gt)  # Close the figure after saving

    # Create a new figure for the remaining images (Observation + Models)
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))

    # Flatten the array of axes for easier iteration
    axs = axs.flatten()

    # Define a consistent font size for titles
    title_fontsize = 14

    # Display the observation image
    axs[0].imshow(observation_img)
    axs[0].set_title('Observation', fontsize=title_fontsize)
    axs[0].axis('off')

    # Display the model images
    for i, (model, image) in enumerate(model_images.items()):
        axs[i+1].imshow(image)
        axs[i+1].set_title(model, fontsize=title_fontsize)
        axs[i+1].axis('off')

    # Turn off any unused axes
    for j in range(len(model_images) + 1, len(axs)):
        axs[j].axis('off')

    # Adjust spacing between the subplots
    plt.subplots_adjust(wspace=0.1, hspace=0.3)

    # Save the comparison figure with high resolution
    plt.savefig(f'comparison_models_image_{
                image_id}.png', dpi=300, bbox_inches='tight')

    # Show the plot for inspection
    plt.show()


# Example usage
visualize_model_comparisons(image_id=16)

In [ ]:
# prompt: Get the metrics for image_id=16 from metrics_results.csv

import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/metrics_results.csv')

# Filter the DataFrame for image_id = 16
image_16_metrics = df[df['Image_ID'] == 16]

# Print the metrics for image_id = 16
print(image_16_metrics)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Read the CSV file
df = pd.read_csv('/content/metrics_results.csv')

# Define metrics where higher values are better
higher_is_better_metrics = ['PSNR', 'SSIM', 'NRQM']

# Define metrics where lower values are better
lower_is_better_metrics = ['LPIPS', 'NIQE', 'PI']

# Normalize metrics
scaler = MinMaxScaler()

# Apply normalization based on whether higher or lower values are better
for metric in higher_is_better_metrics + lower_is_better_metrics:
    df_metric = df[df['Metric'] == metric]
    df_metric[['Value']] = scaler.fit_transform(df_metric[['Value']])

    if metric in lower_is_better_metrics:
        df_metric[['Value']] = 1 - df_metric[['Value']]

    df.update(df_metric)

# Group by 'Model' and 'Metric' and calculate the mean of 'Value'
aggregated_df = df.groupby(['Model', 'Metric'])['Value'].mean().reset_index()

# Pivot the DataFrame to have 'Metric' as columns and 'Model' as rows
pivot_df = aggregated_df.pivot(index='Metric', columns='Model', values='Value')

# Calculate the difference between ESRGAN and Observation for each metric
pivot_df['diff'] = pivot_df['ESRGAN'] - pivot_df['Observation']

# Calculate the sum of the differences for each image ID
df['diff_sum'] = df.groupby('Image_ID')['Value'].transform(
    lambda x: x.sum() if df['Model'].iloc[0] == 'ESRGAN' else -x.sum())

# Sort the IDs by 'diff_sum' and get the second-largest difference
sorted_diff_df = df.groupby('Image_ID')['diff_sum'].sum().reset_index()
sorted_diff_df = sorted_diff_df.sort_values(by='diff_sum', ascending=False)

# Get the second-largest difference
second_max_diff_id = sorted_diff_df.iloc[1]['Image_ID']

print(f"The ID with the second-largest difference is: {second_max_diff_id}")

In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image


def visualize_model_comparisons(image_id):
    """
    Visualizes the super-resolved images generated by different models, comparing them
    with the observation image and the ground truth image. Saves the visualizations
    as PNG files.

    Parameters:
    -----------
    image_id : int
        The ID of the image to visualize (used for both observation and ground truth images).

    Process:
    1. Load the observation image and ground truth image.
    2. Load the super-resolved images from five different models: DAT, ESRGAN, HAT, SRCNN, and SwinIR.
    3. Display the ground truth image individually and save it.
    4. Create a 2x3 grid to visualize the observation image alongside the super-resolved images from each model.
    5. Save the comparison as a PNG image and display the plot.
    """
    # Define the paths to the different folders
    observation_path = 'your/path/here/Dataset/observation_test_images'
    ground_truth_path = 'your/path/here/Dataset/ground_truth_test_images'
    model_paths = {
        'DAT': 'your/path/here/Images/DAT',
        'ESRGAN': 'your/path/here/Images/ESRGAN',
        'HAT': 'your/path/here/Images/HAT',
        'SRCNN': 'your/path/here/Images/SRCNN',
        'SwinIR': 'your/path/here/Images/SwinIR'
    }

    # Find and load the observation image
    observation_image = os.path.join(
        observation_path, f'observation_test_{image_id:04d}.bmp')
    observation_img = Image.open(observation_image)

    # Find and load the ground truth image
    ground_truth_image = os.path.join(
        ground_truth_path, f'ground_truth_test_{image_id:04d}.bmp')
    ground_truth_img = Image.open(ground_truth_image)

    # Find and load the model-generated images
    model_images = {}
    for model, model_path in model_paths.items():
        model_image = os.path.join(
            model_path, f'observation_test_{image_id:04d}.bmp')
        model_images[model] = Image.open(model_image)

    # Save the ground truth image individually
    fig_gt, ax_gt = plt.subplots(figsize=(5, 5))
    ax_gt.imshow(ground_truth_img)
    ax_gt.set_title('Ground Truth', fontsize=14)
    ax_gt.axis('off')
    plt.savefig(f'ground_truth_image_{
                image_id}.png', dpi=300, bbox_inches='tight')
    plt.close(fig_gt)  # Close the figure after saving

    # Create a new figure for the remaining images (Observation + Models)
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))

    # Flatten the array of axes for easier iteration
    axs = axs.flatten()

    # Define a consistent font size for titles
    title_fontsize = 14

    # Display the observation image
    axs[0].imshow(observation_img)
    axs[0].set_title('Observation', fontsize=title_fontsize)
    axs[0].axis('off')

    # Display the model images
    for i, (model, image) in enumerate(model_images.items()):
        axs[i+1].imshow(image)
        axs[i+1].set_title(model, fontsize=title_fontsize)
        axs[i+1].axis('off')

    # Turn off any unused axes
    for j in range(len(model_images) + 1, len(axs)):
        axs[j].axis('off')

    # Adjust spacing between the subplots
    plt.subplots_adjust(wspace=0.1, hspace=0.3)

    # Save the comparison figure with high resolution
    plt.savefig(f'comparison_models_image_{
                image_id}.png', dpi=300, bbox_inches='tight')

    # Show the plot for inspection
    plt.show()


# Example usage
visualize_model_comparisons(image_id=0)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Read the CSV file
df = pd.read_csv('/content/metrics_results (2).csv')

# Define metrics where higher values are better
higher_is_better_metrics = ['PSNR', 'SSIM', 'NRQM']

# Define metrics where lower values are better
lower_is_better_metrics = ['LPIPS', 'NIQE', 'PI']

# Normalize metrics
scaler = MinMaxScaler()

# Apply normalization based on whether higher or lower values are better
for metric in higher_is_better_metrics + lower_is_better_metrics:
    df_metric = df[df['Metric'] == metric]
    df_metric[['Value']] = scaler.fit_transform(df_metric[['Value']])

    if metric in lower_is_better_metrics:
        df_metric[['Value']] = 1 - df_metric[['Value']]

    df.update(df_metric)

# Group by 'Model' and 'Metric' and calculate the mean of 'Value'
aggregated_df = df.groupby(['Model', 'Metric'])['Value'].mean().reset_index()

# Pivot the DataFrame to have 'Metric' as columns and 'Model' as rows
pivot_df = aggregated_df.pivot(index='Metric', columns='Model', values='Value')

# Calculate the difference between each model and the 'Observation' model for each metric
for model in pivot_df.columns:
    if model != 'Observation':
        pivot_df[f'diff_{model}'] = pivot_df[model] - pivot_df['Observation']

# Calculate the sum of the absolute differences for each image ID
df['diff_sum'] = df.groupby('Image_ID').apply(lambda x: (
    x['Value'] - x[x['Model'] == 'Observation']['Value'].mean()).abs().sum()).reset_index(level=0, drop=True)

# Find the ID with the smallest difference
min_diff_id = df.loc[df['diff_sum'].idxmin(), 'Image_ID']

print(f"The ID with the smallest difference is: {min_diff_id}")

In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image


def visualize_model_comparisons(image_id):
    """
    Visualizes the super-resolved images generated by different models, comparing them
    with the observation image and the ground truth image. Saves the visualizations
    as PNG files.

    Parameters:
    -----------
    image_id : int
        The ID of the image to visualize (used for both observation and ground truth images).

    Process:
    1. Load the observation image and ground truth image.
    2. Load the super-resolved images from five different models: DAT, ESRGAN, HAT, SRCNN, and SwinIR.
    3. Display the ground truth image individually and save it.
    4. Create a 2x3 grid to visualize the observation image alongside the super-resolved images from each model.
    5. Save the comparison as a PNG image and display the plot.
    """
    # Define the paths to the different folders
    observation_path = 'your/path/here/Dataset/observation_test_images'
    ground_truth_path = 'your/path/here/Dataset/ground_truth_test_images'
    model_paths = {
        'DAT': 'your/path/here/Images/DAT',
        'ESRGAN': 'your/path/here/Images/ESRGAN',
        'HAT': 'your/path/here/Images/HAT',
        'SRCNN': 'your/path/here/Images/SRCNN',
        'SwinIR': 'your/path/here/Images/SwinIR'
    }

    # Define the paths to the different folders
    observation_path = f'{path}/Dataset/observation_test_images'
    ground_truth_path = f'{path}/Dataset/ground_truth_test_images'
    model_paths = {
        'DAT': f'{path}/Images/Observation/DAT',
        'ESRGAN': f'{path}/Images/Observation/ESRGAN',
        'HAT': f'{path}/Images/Observation/HAT',
        'SRCNN': f'{path}/Images/Observation/SRCNN',
        'SwinIR': f'{path}/Images/Observation/SwinIR'
    }

    # Find and load the observation image
    observation_image = os.path.join(
        observation_path, f'observation_test_{image_id:04d}.bmp')
    observation_img = Image.open(observation_image)

    # Find and load the ground truth image
    ground_truth_image = os.path.join(
        ground_truth_path, f'ground_truth_test_{image_id:04d}.bmp')
    ground_truth_img = Image.open(ground_truth_image)

    # Find and load the model-generated images
    model_images = {}
    for model, model_path in model_paths.items():
        model_image = os.path.join(
            model_path, f'observation_test_{image_id:04d}.bmp')
        model_images[model] = Image.open(model_image)

    # Save the ground truth image individually
    fig_gt, ax_gt = plt.subplots(figsize=(5, 5))
    ax_gt.imshow(ground_truth_img)
    ax_gt.set_title('Ground Truth', fontsize=14)
    ax_gt.axis('off')
    plt.savefig(f'ground_truth_image_{
                image_id}.png', dpi=300, bbox_inches='tight')
    plt.close(fig_gt)  # Close the figure after saving

    # Create a new figure for the remaining images (Observation + Models)
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))

    # Flatten the array of axes for easier iteration
    axs = axs.flatten()

    # Define a consistent font size for titles
    title_fontsize = 14

    # Display the observation image
    axs[0].imshow(observation_img)
    axs[0].set_title('Observation', fontsize=title_fontsize)
    axs[0].axis('off')

    # Display the model images
    for i, (model, image) in enumerate(model_images.items()):
        axs[i+1].imshow(image)
        axs[i+1].set_title(model, fontsize=title_fontsize)
        axs[i+1].axis('off')

    # Turn off any unused axes
    for j in range(len(model_images) + 1, len(axs)):
        axs[j].axis('off')

    # Adjust spacing between the subplots
    plt.subplots_adjust(wspace=0.1, hspace=0.3)

    # Save the comparison figure with high resolution
    plt.savefig(f'comparison_models_image_{
                image_id}.png', dpi=300, bbox_inches='tight')

    # Show the plot for inspection
    plt.show()


# Example usage
visualize_model_comparisons(image_id=1)

In [ ]:
# prompt: Get the metrics for image_id=1 from metrics_results.csv

import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/metrics_results (2).csv')

# Filter the DataFrame for image_id = 1
image_1_metrics = df[df['Image_ID'] == 1]

# Print the metrics for image_id = 1
print(image_1_metrics)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Read the CSV file
df = pd.read_csv('/content/metrics_results (2).csv')

# Define metrics where higher values are better
higher_is_better_metrics = ['PSNR', 'SSIM', 'NRQM']

# Define metrics where lower values are better
lower_is_better_metrics = ['LPIPS', 'NIQE', 'PI']

# Normalize metrics
scaler = MinMaxScaler()

# Apply normalization based on whether higher or lower values are better
for metric in higher_is_better_metrics + lower_is_better_metrics:
    df_metric = df[df['Metric'] == metric]
    df_metric[['Value']] = scaler.fit_transform(df_metric[['Value']])

    if metric in lower_is_better_metrics:
        df_metric[['Value']] = 1 - df_metric[['Value']]

    df.update(df_metric)

# Create a DataFrame to store whether Observation is better for each ID
better_observation_df = pd.DataFrame()

# Iterate over each unique Image_ID
for image_id in df['Image_ID'].unique():
    # Filter data for the current Image_ID
    id_df = df[df['Image_ID'] == image_id]

    # Create a pivot table to compare Observation with other models
    pivot_df = id_df.pivot(index='Metric', columns='Model', values='Value')

    if 'Observation' in pivot_df.columns:
        # Check if Observation is better than all other models for each metric
        all_better = True
        for model in pivot_df.columns:
            if model != 'Observation':
                # Check if Observation is better than this model for all metrics
                if not ((pivot_df['Observation'] >= pivot_df[model]).all() or
                        (pivot_df['Observation'] <= pivot_df[model]).all()):
                    all_better = False
                    break

        # Record the result
        better_observation_df = pd.concat([better_observation_df, pd.DataFrame(
            {'Image_ID': [image_id], 'Observation_Better': [all_better]})], ignore_index=True)

# Find IDs where Observation is better than all models
better_observation_ids = better_observation_df[better_observation_df['Observation_Better']]['Image_ID']

print(f"IDs where Observation wins against all models: {
      list(better_observation_ids)}")